In [ ]:
import argparse

import torch.distributed as dist
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

import test  # import test.py to get mAP after each epoch
from models import *
from utils.datasets import *
from utils.utils import *

from mymodel import *

# Hyperparameters (results68: 59.9 mAP@0.5 yolov3-spp-416) https://github.com/ultralytics/yolov3/issues/310

hyp = {'giou': 3.54,  # giou loss gain
       'cls': 37.4,  # cls loss gain
       'cls_pw': 1.0,  # cls BCELoss positive_weight
       'obj': 64.3,  # obj loss gain (*=img_size/320 if img_size != 320)
       'obj_pw': 1.0,  # obj BCELoss positive_weight
       'iou_t': 0.225,  # iou training threshold
       'lr0': 0.01,  # initial learning rate (SGD=5E-3, Adam=5E-4)
       'lrf': -4.,  # final LambdaLR learning rate = lr0 * (10 ** lrf)
       'momentum': 0.937,  # SGD momentum
       'weight_decay': 0.000484,  # optimizer weight decay
       'fl_gamma': 0.5,  # focal loss gamma
       'hsv_h': 0.0138,  # image HSV-Hue augmentation (fraction)
       'hsv_s': 0.678,  # image HSV-Saturation augmentation (fraction)
       'hsv_v': 0.36,  # image HSV-Value augmentation (fraction)
       'degrees': 1.98,  # image rotation (+/- deg)
       'translate': 0.05,  # image translation (+/- fraction)
       'scale': 0.05,  # image scale (+/- gain)
       'shear': 0.641}  # image shear (+/- deg)


In [ ]:
parser = argparse.ArgumentParser()

parser.add_argument('--batch-size', type=int, default=16)  # effective bs = batch_size * accumulate = 16 * 4 = 64
parser.add_argument('--accumulate', type=int, default=4, help='batches to accumulate before optimizing')
parser.add_argument('--cfg', type=str, default='cfg/yolov3-tiny-1cls_1.cfg', help='*.cfg path')
parser.add_argument('--data', type=str, default='data/coco2017.data', help='*.data path')
parser.add_argument('--img-size', nargs='+', type=int, default=[320], help='train and test image-sizes')
parser.add_argument('--rect', action='store_true', help='rectangular training')
parser.add_argument('--cache-images', action='store_true', help='cache images for faster training')
parser.add_argument('--weights', type=str, default='/home/denggc/DAC2021/dgc/April/ultra_bypass/weights/test_best.pt', help='initial weights path')
parser.add_argument('--arc', type=str, default='default', help='yolo architecture')  # default, uCE, uBCE
parser.add_argument('--name', default='', help='renames results.txt to results_name.txt if supplied')
parser.add_argument('--device', default='1', help='device id (i.e. 0 or 0,1 or cpu)')
parser.add_argument('--single-cls', action='store_true', help='train as single-class dataset')
parser.add_argument('--var', type=float, help='debug variable')
opt = parser.parse_known_args()[0]
print(opt)
print(opt.weights)
device = torch_utils.select_device(opt.device, batch_size=opt.batch_size)
print(device)


img_size, img_size_test = opt.img_size if len(opt.img_size) == 2 else opt.img_size * 2  # train, test sizes
batch_size = opt.batch_size
accumulate = opt.accumulate  # effective bs = batch_size * accumulate = 16 * 4 = 64
weights = opt.weights  # initial training weights


test_path = '../DAC-SDC2021/dataset/sample'
nc = 1 

In [ ]:
model = UltraNet_Bypass().to(device)
model.hyp = hyp
model.nc = 1
model.arc = 'default'

In [ ]:
if weights.endswith('.pt'):  # pytorch format
    # possible weights are '*.pt', 'yolov3-spp.pt', 'yolov3-tiny.pt' etc.
    print("load weights...")
    model.load_state_dict(torch.load(weights, map_location=device)['model'])

In [ ]:
batch_size = min(batch_size, 1)
nw = min([os.cpu_count(), batch_size if batch_size > 1 else 0, 8])  # number of workers
dataset = LoadImagesAndLabels(test_path, img_size_test, batch_size * 2,
                              hyp=hyp,
                              rect=False,
                              cache_images=opt.cache_images,
                              single_cls=opt.single_cls)
testloader = torch.utils.data.DataLoader(dataset,
                                         batch_size=batch_size * 2,
                                         num_workers=nw,
                                         pin_memory=True,
                                         collate_fn=dataset.collate_fn)

In [ ]:
results = test.test(opt.cfg,
                    opt.data,
                    batch_size=batch_size * 2,
                    img_size=img_size_test,
                    model=model,
                    conf_thres=0.001,  # 0.001 if opt.evolve or (final_epoch and is_coco) else 0.01,
                    iou_thres=0.6,
                    save_json=False,
                    single_cls=opt.single_cls,
                    dataloader=testloader)
print(results)